In [1]:
from keras.models import load_model  # TensorFlow is required for Keras to work

import cv2
import numpy as np
import os 

# RUN MASSAL DENGAN KLASIFIKASI JENIS MOTOR TERLEBIH DAHULU

In [20]:
# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

folder_path = "images/roi-motor/"

vario = 0
beat = 0
bebek = 0
nmax = 0
rx = 0
scoopy = 0

Exception ignored in: <function UniquePtr.__del__ at 0x000001EAA0102F70>
Traceback (most recent call last):
  File "C:\Users\whisn\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\framework\c_api_util.py", line 71, in __del__
    obj = self._obj
KeyboardInterrupt: 


In [ ]:
# # Mengakses seluruh subfolder dalam folder "roi-motor"
# for root, dirs, files in os.walk(folder_path):
#     for dir_name in dirs:
#         subfolder_path = os.path.join(root, dir_name)
#         # print(f"dir_name: {dir_name}")
#         file_counter = 0
#         # Membaca isi subfolder
#         for file_name in os.listdir(subfolder_path):
#             file_path = os.path.join(subfolder_path, file_name)
#             # print(f"{file_name}")
#             # Lakukan sesuatu dengan file
#             # Contoh: Membaca file gambar
#             if file_name.endswith('.jpg'):
#                 image = cv2.imread(file_path)
#                 boundingImage = cv2.imread(file_path)

#                 # Disable scientific notation for clarity
#                 np.set_printoptions(suppress=True)

#                 # Resize the raw image into (224-height,224-width) pixels
#                 image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

#                 # Make the image a numpy array and reshape it to the models input shape.
#                 image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

#                 # Normalize the image array
#                 image = (image / 127.5) - 1
#                 prediction = model.predict(image)

#                 index = np.argmax(prediction)
#                 class_name = class_names[index][:1]
#                 confidence_score = prediction[0][index]
#                 # print(class_name)
#                 if int(class_name) == 0:
#                     beat += 1
#                     print("beat")
#                 elif int(class_name) == 1:
#                     vario += 1
#                     print("vario")
#                 elif int(class_name) == 2:
#                     bebek += 1
#                     print("bebek")
#                 elif int(class_name) == 3:
#                     nmax += 1
#                     print("nmax")
#                 elif int(class_name) == 4:
#                     rx += 1
#                     print("rx")
#                 elif int(class_name) == 5:
#                     scoopy += 1
#                     print("scoopy")

                    
# print(f"vario: {vario}\tbebek: {bebek}\tscoopy: {scoopy}\tnmax: {nmax}\tbeat: {beat}\trx: {rx}")
# print(f"total: {scoopy+vario+nmax+rx+beat+bebek}")

In [ ]:
# Mengakses seluruh subfolder dalam folder "roi-motor"
for root, dirs, files in os.walk(folder_path):
    for dir_name in dirs:
        subfolder_path = os.path.join(root, dir_name)
        # print(f"dir_name: {dir_name}")
        file_counter = 0
        # Membaca isi subfolder
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)
            # print(f"{file_name}")
            # Lakukan sesuatu dengan file
            # Contoh: Membaca file gambar
            if file_name.endswith('.jpg'):
                image = cv2.imread(file_path)
                boundingImage = cv2.imread(file_path)

                # Disable scientific notation for clarity
                np.set_printoptions(suppress=True)

                # Resize the raw image into (224-height,224-width) pixels
                image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

                # Make the image a numpy array and reshape it to the models input shape.
                image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

                # Normalize the image array
                image = (image / 127.5) - 1
                prediction = model.predict(image)

                index = np.argmax(prediction)
                class_name = class_names[index][:1]
                confidence_score = prediction[0][index]

                frame = cv2.resize(boundingImage, (boundingImage.shape[1]*4, boundingImage.shape[0]*4))
                hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

                lower_white = np.array([0, 0, 200])
                upper_white = np.array([180, 30, 255])
                mask = cv2.inRange(hsv_frame, lower_white, upper_white)

                result = cv2.bitwise_and(frame, frame, mask=mask)
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for contour in contours:
                    area = cv2.contourArea(contour)
                    x, y, w, h = cv2.boundingRect(contour)
                    if int(class_name) == 0 : # todo BEAT
                        if y > 120:
                            if (area > 1500 and h/w > 1.6) or (area > 1500 and area < 4800 and h/w < 1 and h/w > 0.65) or (area > 2500 and area < 7000 and h/w > 1 and h/w < 1.5) or (area > 13500 and h/w > 0.99 and h/w < 1.25 and area < 27000) or (area > 30000 and h/w > 1.15):
                                cv2.rectangle(boundingImage, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    elif int(class_name) == 1 : # Todo VARIO
                        if y > 50 and area < 5000 :
                            if (area > 500 and h/w >= 1 and h/w < 1.6) or (area > 500 and h/w > 3.5) or (area > 3000 and h/w < 1 and h/w > 0.6):
                                cv2.rectangle(boundingImage, (x, y), (x+w, y+h), (0, 255, 0), 2)         
                    elif int(class_name) == 2 : # todo BEBEK
                        if (area > 1500 and h/w > 0.6 and h/w < 1.1 and h/w != 1) or (y<50 and h/w < 0.6 and area > 1000 and area < 2500 and h/w != 1) or (y<50 and h/w < 0.6 and h/w < 1 and area > 2500  and h/w != 1) :
                            cv2.rectangle(boundingImage, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    elif int(class_name) == 3 : # todo NMAX
                        if area > 200 and area < 1000  and h/w > 0.5 and h/w < 0.65:
                            cv2.rectangle(boundingImage, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    elif int(class_name) == 4 : # todo RX
                        if area > 2000 and h/w < 1.6:
                            cv2.rectangle(boundingImage, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    elif int(class_name) == 5 : # todo SCOOPY
                        ratio_y2_per_h = (y+h)/h
                        if area > 4000 and area < 18000 and y > 50 and h/w > 0.5 and ratio_y2_per_h < 4.5:
                            cv2.rectangle(boundingImage, (x, y), (x+w, y+h), (0, 255, 0), 2)

                    cv2.imshow("bounding Image", boundingImage)
                    if cv2.waitKey(1) &0xFF == 27:
                        break
                    

print(f"vario: {vario}\tbebek: {bebek}\tscoopy: {scoopy}\tnmax: {nmax}\tbeat: {beat}\trx: {rx}")
print(f"total: {scoopy+vario+nmax+rx+beat+bebek}")

1/1 [==============================] - 0s 34ms/step
vario: 0	bebek: 0	scoopy: 0	nmax: 0	beat: 0	rx: 0
total: 0


In [ ]:
# folder_path_scoopy = 'images/roi-motor/SCOOPY'  # Ubah sesuai dengan path folder foto
# folder_path_beat = 'images/roi-motor/BEAT'  # Ubah sesuai dengan path folder foto
# folder_path_bebek = 'images/roi-motor/BEBEK'  # Ubah sesuai dengan path folder foto
# folder_path_nmax = 'images/roi-motor/NMAX'  # Ubah sesuai dengan path folder foto
# folder_path_rx = 'images/roi-motor/RX'  # Ubah sesuai dengan path folder foto
# folder_path_vario = 'images/roi-motor/VARIO'  # Ubah sesuai dengan path folder foto

# image_files = os.listdir(folder_path_vario)
# count = 0

# VARIO

In [16]:
folder_path_vario = 'images/roi-motor/VARIO'  # Ubah sesuai dengan path folder foto
image_files = os.listdir(folder_path_vario)
count = 0

for image_file in image_files:
    image_path = os.path.join(folder_path_vario, image_file)
    frame = cv2.imread(image_path)

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow("putih", frame)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    result = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("result", result)
    # Temukan kontur pada gambar hasil
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gambar kotak pada setiap kontur
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        if y > 50 and area < 5000:
            if (area > 500 and h/w >= 1 and h/w < 1.6) or (area > 500 and h/w > 3.5) or (area > 3000 and h/w < 1 and h/w > 0.6):
                count+=1
                print(f"num {count} x:{x} y:{y} w:{w} h:{h} area:{area} ratio:{h/w}")
                cv2.putText(frame, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.imwrite("result/roi-lampu/fuzzy/vario/" + str(count)+'.jpg', frame[y:y+h, x:x+w])

    cv2.imshow('Hasil Deteksi', frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

print(count)
cv2.destroyAllWindows()

num 1 x:121 y:233 w:35 h:35 area:723.5 ratio:1.0
num 2 x:58 y:216 w:32 h:46 area:783.5 ratio:1.4375
num 3 x:157 y:180 w:47 h:52 area:560.5 ratio:1.1063829787234043
num 4 x:43 y:135 w:18 h:77 area:848.0 ratio:4.277777777777778
num 5 x:53 y:211 w:32 h:46 area:794.0 ratio:1.4375
num 6 x:116 y:169 w:87 h:94 area:1348.0 ratio:1.0804597701149425
num 7 x:37 y:127 w:19 h:81 area:933.0 ratio:4.2631578947368425
num 8 x:116 y:229 w:76 h:94 area:1694.0 ratio:1.236842105263158
num 9 x:35 y:205 w:131 h:104 area:4653.0 ratio:0.7938931297709924
num 10 x:109 y:187 w:79 h:95 area:1742.0 ratio:1.2025316455696202
num 11 x:107 y:180 w:80 h:96 area:1887.5 ratio:1.2
num 12 x:129 y:242 w:80 h:98 area:1913.0 ratio:1.225
num 13 x:123 y:248 w:85 h:104 area:2037.0 ratio:1.223529411764706
num 14 x:135 y:226 w:89 h:109 area:2087.5 ratio:1.2247191011235956
num 15 x:44 y:184 w:36 h:132 area:1862.0 ratio:3.6666666666666665
num 16 x:140 y:247 w:90 h:111 area:2186.0 ratio:1.2333333333333334
num 17 x:58 y:320 w:32 h:48 a

# NMAX

In [14]:
folder_path_nmax = 'images/roi-motor/NMAX'  # Ubah sesuai dengan path folder foto
image_files = os.listdir(folder_path_nmax)
count = 0

for image_file in image_files:
    image_path = os.path.join(folder_path_nmax, image_file)
    frame = cv2.imread(image_path)

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow("putih", frame)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    result = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("result", result)
    # Temukan kontur pada gambar hasil
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gambar kotak pada setiap kontur
    for contour in contours:
        area = cv2.contourArea(contour)
        count+=1
        x, y, w, h = cv2.boundingRect(contour)
        if area > 200 and area < 1000  and h/w > 0.5 and h/w < 0.65:
            print(f"num {count} x:{x} y:{y} w:{w} h:{h} area:{area} ratio:{h/w}")
            cv2.putText(frame, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.imwrite("result/roi-lampu/fuzzy/nmax/" + str(count)+'.jpg', frame[y:y+h, x:x+w])

    cv2.imshow('Hasil Deteksi', frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

num 4 x:103 y:206 w:32 h:17 area:286.5 ratio:0.53125
num 7 x:51 y:186 w:33 h:20 area:404.0 ratio:0.6060606060606061
num 13 x:77 y:118 w:37 h:21 area:416.5 ratio:0.5675675675675675


# RX


In [12]:
folder_path_rx = 'images/roi-motor/RX'  # Ubah sesuai dengan path folder foto
image_files = os.listdir(folder_path_rx)
count = 0

for image_file in image_files:
    image_path = os.path.join(folder_path_rx, image_file)
    frame = cv2.imread(image_path)

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow("putih", frame)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    result = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("result", result)
    # Temukan kontur pada gambar hasil
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gambar kotak pada setiap kontur
    for contour in contours:
        area = cv2.contourArea(contour)
        count+=1
        x, y, w, h = cv2.boundingRect(contour)
        if area > 2000 and h/w < 1.6:
            print(f"num {count} x:{x} y:{y} w:{w} h:{h} area:{area} ratio:{h/w}")
            cv2.putText(frame, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.imwrite("result/roi-lampu/fuzzy/rx/" + str(count)+'.jpg', frame[y:y+h, x:x+w])

    cv2.imshow('Hasil Deteksi', frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

num 12 x:66 y:116 w:74 h:69 area:2556.0 ratio:0.9324324324324325
num 33 x:64 y:108 w:73 h:70 area:2701.0 ratio:0.958904109589041
num 59 x:73 y:105 w:75 h:75 area:2847.0 ratio:1.0
num 76 x:76 y:205 w:68 h:71 area:3390.5 ratio:1.0441176470588236
num 81 x:77 y:105 w:89 h:96 area:3913.5 ratio:1.0786516853932584
num 97 x:84 y:231 w:71 h:78 area:3864.5 ratio:1.0985915492957747
num 103 x:117 y:129 w:59 h:69 area:3163.5 ratio:1.1694915254237288
num 115 x:78 y:219 w:75 h:82 area:4237.0 ratio:1.0933333333333333
num 122 x:114 y:114 w:61 h:71 area:3286.0 ratio:1.1639344262295082
num 134 x:84 y:262 w:78 h:89 area:4763.0 ratio:1.141025641025641
num 140 x:121 y:155 w:62 h:73 area:3510.5 ratio:1.1774193548387097


# BEAT

In [11]:
folder_path_beat = 'images/roi-motor/BEAT'  # Ubah sesuai dengan path folder foto
image_files = os.listdir(folder_path_beat)
count = 0

for image_file in image_files:
    image_path = os.path.join(folder_path_beat, image_file)
    frame = cv2.imread(image_path)

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow("putih", frame)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    result = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("result", result)
    # Temukan kontur pada gambar hasil
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gambar kotak pada setiap kontur
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        if y > 120:
            if (area > 1500 and h/w > 1.6) or (area > 1500 and area < 4800 and h/w < 1 and h/w > 0.65) or (area > 2500 and area < 7000 and h/w > 1 and h/w < 1.5) or (area > 13500 and h/w > 0.99 and h/w < 1.25 and area < 27000) or (area > 30000 and h/w > 1.15):
                count+=1   
                print(f"num {count} x:{x} y:{y} w:{w} h:{h} area:{area} ratio:{h/w}")
                cv2.putText(frame, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.imwrite("result/roi-lampu/fuzzy/beat/" + str(count)+'.jpg', frame[y:y+h, x:x+w])

    cv2.imshow('Hasil Deteksi', frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

num 1 x:50 y:297 w:82 h:136 area:2974.5 ratio:1.6585365853658536


# UNTUK BEBEK

In [17]:
folder_path_bebek = 'images/roi-motor/BEBEK'  # Ubah sesuai dengan path folder foto
image_files = os.listdir(folder_path_bebek)
count = 0

for image_file in image_files:
    image_path = os.path.join(folder_path_bebek, image_file)
    frame = cv2.imread(image_path)

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow("putih", frame)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    result = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("result", result)
    # Temukan kontur pada gambar hasil
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gambar kotak pada setiap kontur
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        # if area > 1000 :
        if (area > 1500 and h/w > 0.6 and h/w < 1.1 and h/w != 1) or (y<50 and h/w < 0.6 and area > 1000 and area < 2500 and h/w != 1) or (y<50 and h/w < 0.6 and h/w < 1 and area > 2500  and h/w != 1) :
            count+=1    
            print(f"num {count} x:{x} y:{y} w:{w} h:{h} area:{area} ratio:{h/w}")
            cv2.putText(frame, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.imwrite("result/roi-lampu/fuzzy/bebek/" + str(count)+'.jpg', frame[y:y+h, x:x+w])

    cv2.imshow('Hasil Deteksi', frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

num 1 x:117 y:77 w:82 h:74 area:3369.0 ratio:0.9024390243902439
num 2 x:117 y:81 w:84 h:75 area:3434.5 ratio:0.8928571428571429
num 3 x:119 y:94 w:88 h:78 area:3752.5 ratio:0.8863636363636364
num 4 x:121 y:112 w:90 h:81 area:3954.5 ratio:0.9
num 5 x:127 y:125 w:92 h:85 area:4120.0 ratio:0.9239130434782609
num 6 x:40 y:313 w:124 h:88 area:2738.5 ratio:0.7096774193548387
num 7 x:88 y:180 w:95 h:60 area:2844.5 ratio:0.631578947368421
num 8 x:49 y:260 w:129 h:94 area:2866.0 ratio:0.7286821705426356
num 9 x:97 y:123 w:99 h:61 area:3079.0 ratio:0.6161616161616161
num 10 x:48 y:256 w:138 h:98 area:3019.5 ratio:0.7101449275362319
num 11 x:101 y:105 w:103 h:67 area:3426.0 ratio:0.6504854368932039
num 12 x:91 y:93 w:71 h:55 area:1824.0 ratio:0.7746478873239436
num 13 x:89 y:60 w:71 h:57 area:1867.0 ratio:0.8028169014084507
num 14 x:88 y:128 w:75 h:59 area:2015.0 ratio:0.7866666666666666
num 15 x:116 y:116 w:83 h:69 area:2560.5 ratio:0.8313253012048193
num 16 x:105 y:70 w:70 h:56 area:1848.0 rati

## UNTUK SCOOPY

In [19]:
folder_path_scoopy = 'images/roi-motor/SCOOPY'  # Ubah sesuai dengan path folder foto
image_files = os.listdir(folder_path_scoopy)
count = 0

for image_file in image_files:
    image_path = os.path.join(folder_path_scoopy, image_file)
    frame = cv2.imread(image_path)

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow("putih", frame)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    result = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("result", result)
    # Temukan kontur pada gambar hasil
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gambar kotak pada setiap kontur
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)  
        ratio_y2_per_h = (y+h)/h
        if area > 4000 and area < 18000 and y > 50 and h/w > 0.5 and ratio_y2_per_h < 4.5:
            count += 1
            print(f"num {count} x:{x} y:{y} w:{w} h:{h} area:{area} ratio:{h/w} y2/h: {ratio_y2_per_h} y2-y1:{(y+h)-y}")
            cv2.putText(frame, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.imwrite("result/roi-lampu/fuzzy/scoopy/" + str(count)+'.jpg', frame[y:y+h, x:x+w])

    cv2.imshow('Hasil Deteksi', frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

num 1 x:50 y:273 w:99 h:116 area:7783.5 ratio:1.1717171717171717 y2/h: 3.353448275862069 y2-y1:116
num 2 x:49 y:291 w:102 h:120 area:7940.0 ratio:1.1764705882352942 y2/h: 3.425 y2-y1:120
num 3 x:48 y:285 w:143 h:123 area:9184.0 ratio:0.8601398601398601 y2/h: 3.317073170731707 y2-y1:123
num 4 x:48 y:299 w:109 h:122 area:8499.5 ratio:1.1192660550458715 y2/h: 3.4508196721311477 y2-y1:122
